<a href="https://colab.research.google.com/github/Miragecore/SandBox/blob/devel/fastapi_Image_streamming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Opencv Video Steam using FastAPI

참조 1 Flask, Opencv 스트리밍 서버  
https://pyimagesearch.com/2019/09/02/opencv-stream-video-to-web-browser-html-page/

참조 2  Flask -> fastAPI  
https://testdriven.io/blog/moving-from-flask-to-fastapi/

참조 3 rtsp 스트리밍  
https://github.com/mpimentel04/rtsp_fastapi

TODO :  
- 끝나지 않는 데몬은 어떻게 처리하지?

In [3]:
#스틸컷에 쓸 이미지 로딩을 위한 구글드라이브 마운트
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import numpy as np
import imutils
import cv2
from imutils.video import VideoStream
import threading
import time
from google.colab.patches import cv2_imshow

In [ ]:
import os
#디렉토리 이동
root_path = '/content/drive/MyDrive'
paths = ['MLOps_Coursera', 'ML_for_Production', 'Course1', 'Week1','images']

os.chdir(root_path)

for folder in paths:
  if not os.path.isdir(folder):
    os.mkdir(folder)
    print('mkdir {folder}')
  os.chdir(folder)

print(os.getcwd())

image_files = [
    'apple.jpg',
    'clock.jpg',
    'oranges.jpg',
    'car.jpg'
]

images = [] 
#스트리밍할 이미지 로딩
for image_file in image_files:
  timg = cv2.imread(image_file)
  timg = imutils.resize(timg, width=400)
  images.append(timg)

In [6]:
outputframe = None
lock = threading.Lock()
vs = VideoStream(src=0).start()

def generate_frame():
  global outputframe, lock

  i = 0;
  while True:
    #다른 사진으로 프레임 변경
    frame = images[i];
    i += 1

    with lock:
      outputframe = frame.copy()
    
    if i >= len(images):
      i = 0
    #FPS를 1로
    time.sleep(1)
    

In [7]:
def generate():
  global outputframe, lock

  while True:
    with lock:
      if outputframe is None:
        continue

      (flag, encodedImage) = cv2.imencode(".jpg", outputframe)

      if flag is None:
        continue

    yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' +
			  bytearray(encodedImage) + b'\r\n')

In [ ]:
!pip install fastapi

In [8]:
from fastapi import FastAPI
from fastapi.responses import StreamingResponse, HTMLResponse

In [9]:
app = FastAPI(title='Opencv Streaming')

@app.get("/",response_class=HTMLResponse)
async def home():
  return """
  <html>
  <head>
    <title>Image Files Streaming</title>
  </head>
  <body>
    <h1>Image File Streaming</h1>
    <img src="/video_feed">
  </body>
</html>
"""

@app.get("/video_feed")
def video_feed():
	# return the response generated along with the specific media
	# type (media_type)
	return StreamingResponse(generate(),media_type="multipart/x-mixed-replace;boundary=frame")

In [ ]:
!pip install pyngrok
!pip install uvicorn

In [ ]:
#ngrok Auth
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

!/ngrok authtoken "24hROn7coCpNvy220TtwTSP0aYA_3ZaXPFsPZy4FprkJNwR35"

In [11]:
#start frame generate thread
t = threading.Thread(target=generate_frame,args=())
t.daemon = True
t.start()

In [12]:
# Allows the server to be run in this interactive environment
#nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
#host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server!    
#uvicorn.run(app, host=host, port=8000)
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [2291]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://33ac-34-125-234-163.ngrok.io
INFO:     14.35.26.238:0 - "GET / HTTP/1.1" 200 OK
INFO:     14.35.26.238:0 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     14.35.26.238:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2291]
